In [47]:
import gspread
import mysqlcredentials as mc
import mysql.connector
from oauth2client.service_account import ServiceAccountCredentials

In [48]:
# initialize variables for gspread
scope = ['https://spreadsheets.google.com/feeds',
'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('GoogleSheetsToMySQL.json', scope)
client = gspread.authorize(creds)

In [49]:
# define method to pull data from spreadsheet
def GetSpreadsheetData(sheetName, worksheetIndex):
    sheet = client.open(sheetName).get_worksheet(worksheetIndex)
    return sheet.get_all_values()[1:]

In [50]:
def PreserveNULLValues(listName):
    print('Preserving NULL values…')
    for x in range(len(listName)):
        for y in range(len(listName[x])):
            if listName[x][y] == '':
                listName[x][y] = None
    print('NULL values preserved.')

In [51]:
data=GetSpreadsheetData('db_aircall', 2)

In [52]:
print(data[0])
print(len(data))

['CAdde3ac5bf4be8bb3b6ad9539b0b90555', '', 'Inbound - Abandoned', '', 'short_abandoned', '5/6/2024 19:21:54', '5/6/2024 19:21:55', '5/6/2024 19:21:55', '1', '', '', '', '[No associated user]', '', '', '', '', '', '', '', '']
50724


In [53]:
def insert_update_call(call_data):
    
    try:
        connection = mysql.connector.connect(
        user = mc.user,
        password = mc.password,
        host = mc.host,
        database = mc.database
        )
        
        drop_table = " DROP TABLE IF EXISTS aircalls_calls "
        create_table = """ CREATE TABLE  aircalls_calls (
                    id INT,
                    direct_link VARCHAR(255),
                    direction VARCHAR(255),
                    status VARCHAR(255),
                    missed_call_reason VARCHAR(255),
                    started_at DATETIME,
                    answered_at DATETIME,
                    ended_at DATETIME,
                    duration INT,
                    voicemail BOOLEAN,
                    asset VARCHAR(255),
                    raw_digits VARCHAR(255),
                    user_id INT,
                    archived BOOLEAN,
                    assigned_to INT,
                    transferred_by INT,
                    transferred_to INT,
                    number_id INT,
                    cost FLOAT,
                    country_code_a2 VARCHAR(255),
                    pricing_type VARCHAR(255)
                    ); """
        
        script_insert_or_update_calls = """
        INSERT INTO aircalls_calls (id, direct_link, direction, status, missed_call_reason, started_at, answered_at, ended_at, duration, voicemail, asset, raw_digits, user_id, archived, assigned_to, transferred_by, transferred_to, number_id, cost, country_code_a2, pricing_type) 
        VALUES (%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s);
                    """                              
        cursor = connection.cursor()
        cursor.execute(drop_table)
        print(f"Call dropped")
        cursor.execute(create_table)
        print(f"Call Table Created")
              
        for i in data:
            cursor.execute(script_insert_or_update_calls, i)
        
        connection.commit()
        print(f"Call Updated")
        #print(f"Call Values: {values}")

    except mysql.connector.Error as error :
        connection.rollback()
        print("Error: {}. Table {} not updated!".format(error,'aircalls_calls'))
# We need to close the cursor and the connection, and this needs to be done regardless of what happened above.
    finally:
        cursor.execute('SELECT COUNT(*) FROM {}'.format('aircalls_calls'))
        rowCount = cursor.fetchone()[0]
        print('aircall_calls', 'row count:', rowCount)
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed.")        
    

In [54]:
insert_update_call(data)

Call dropped
Call Table Created
aircall_calls row count: 44265
MySQL connection is closed.


KeyboardInterrupt: 